In [3]:
## import libraries

from requests import get
import numpy as np
import pandas as pd
import time 
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
from tqdm import tqdm

In [4]:
## assign constants

page_link = "https://www.ivi.ru/collections/best-movies/"
url = "https://www.ivi.ru"

In [5]:
## Getting links

def get_page(page_link):
    response = get(page_link, headers={"User-Agent":UserAgent().chrome})
    return response.content

def parse_page_content(content,links):
    soup = BeautifulSoup(content,'html.parser')
    a = soup.find_all("a",{"class":"nbl-slimPosterBlock nbl-slimPosterBlock_type_poster nbl-slimPosterBlock_iconStatus_none nbl-slimPosterBlock_available collections__nbl-slimPosterBlock"})
    for item in a:
        links.append(item.get("href"))

def scroll_page(page_link):
    page_number = 1
    links = []
    while True:
        new_url = page_link + f'page{page_number}?sort=pop'
        content = get_page(new_url)
        parse_page_content(content,links)
        page_number += 1
        if page_number == 5:
            break
    return links


In [6]:
## links to the best films
links = scroll_page(page_link)
links[:10]

['/watch/516421',
 '/watch/109726',
 '/watch/100130',
 '/watch/210332',
 '/watch/87485',
 '/watch/515403',
 '/watch/33560',
 '/watch/titanik',
 '/watch/100018',
 '/watch/89599']

In [7]:
len(links)

120

In [8]:
def convert_into_minute(s):
    try:
        s = s.split(" ")
        k = []
        for i in s:
            if i == "ч.":
                h = 60
            elif i == "мин.":
                m = 1
            elif i.isnumeric():
                k.append(int(i))

        if len(k) == 2:
            s = k[0]*h + k[1]*m
        else:
            s = k[0]*m
        return s
    except:
        s = " ".join(s)
        return s

In [12]:
## Getting necessary content
def get_content_about_film(url):
    film_name = []
    film_year = []
    film_duration = []
    film_age_restriction = []
    film_country = []
    film_genre = []
    film_rating = []
    film_number_of_reviews = []
    for i in tqdm(links):
        new_url = url + i
        content = get_page(new_url)
        soup = BeautifulSoup(content,'html.parser')
        
        name = soup.find("div",{"class":"watchTitle contentCard__watchTitle"}).text
        slice_index = name.find("(Фильм")
        name = name[:slice_index]
        film_name.append(name)

        year = soup.find("div",{"class":"watchParams__item"}).text
        film_year.append(year)

        duration = soup.find("a",{"class":"nbl-link nbl-link_style_wovou nbl-link nbl-link_noLink"}).text
        duration = convert_into_minute(duration)
        film_duration.append(duration)

        age_restriction = soup.find_all("a",{"class":"nbl-link nbl-link_style_wovou nbl-link nbl-link_noLink"})[1].text
        if "+" not in age_restriction:
            age_restriction = "0+"
        film_age_restriction.append(age_restriction)

        country = soup.find("div",{"class":"watchParams__item watchParams__item_hasDot"}).text
        film_country.append(country)

        genre = soup.find_all("div",{"class":"watchParams__item watchParams__item_hasDot"})[1].text
        film_genre.append(genre)

        rating = float(soup.find("div",{"class":"nbl-ratingPlate__value"}).text.replace(",","."))
        film_rating.append(rating)

        if number_of_reviews := soup.find("div",{"class":"nbl-superscript__text"}):
            number_of_reviews = number_of_reviews.text
        else:
            number_of_reviews = None 
        film_number_of_reviews.append(number_of_reviews)

    data = {"Name":film_name, "Year":film_year, "Duration":film_duration, "Age_restriction":film_age_restriction,
            "Country":film_country, "Genre":film_genre,"Rating":film_rating, "Number_of_reviews":film_number_of_reviews}

    return data
        
        

In [13]:
## creating data frame
def create_data_frame(data):
    df = pd.DataFrame(data=data)
    return df

In [15]:
data = get_content_about_film(url)
df = create_data_frame(data)

100%|██████████| 120/120 [01:40<00:00,  1.19it/s]


In [16]:
df.to_csv("data_before.csv")